## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

# **Data Binning, Cleaning, and Standardizing**

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN'])

**'AFFILIATION' Binning**

In [ ]:
# Look at AFFILIATION value counts for binning
application_df['AFFILIATION'].value_counts()

AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `aff_to_replace`
aff_to_replace = ['Family/Parent', 'National', 'Regional', 'Other']

# Replace in dataframe
for app in aff_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(app,"Other")

# # Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

AFFILIATION
Independent         18480
CompanySponsored    15705
Other                 114
Name: count, dtype: int64

**'INCOME_AMT' Binning**

In [ ]:
# Look at INCOME_AMT value counts for binning
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `income_to_replace`
income_amt_to_replace = ['1M-5M', '10M-50M', '5M-10M', '50M+']

# Replace in dataframe
for app in income_amt_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(app,"1M+")

# # Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M+               1519
1-9999             728
10000-24999        543
Name: count, dtype: int64

**'NAME' Binning**

In [ ]:

# Look at NAME value counts for binning
name_counts_df = application_df['NAME'].value_counts()
name_counts_df = name_counts_df.reset_index()
name_counts_df.columns = ['NAME', 'COUNT']
print(name_counts_df)

                                                    NAME  COUNT
0                                 PARENT BOOSTER USA INC   1260
1                                          TOPS CLUB INC    765
2                     UNITED STATES BOWLING CONGRESS INC    700
3                            WASHINGTON STATE UNIVERSITY    492
4        AMATEUR ATHLETIC UNION OF THE UNITED STATES INC    408
...                                                  ...    ...
19563                      ST LOUIS SLAM WOMENS FOOTBALL      1
19564                    AIESEC ALUMNI IBEROAMERICA CORP      1
19565                              WEALLBLEEDRED ORG INC      1
19566  AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP &...      1
19567                           WATERHOUSE CHARITABLE TR      1

[19568 rows x 2 columns]


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `names_to_replace`
names_to_replace = []

for index, row in name_counts_df.iterrows():
  if row['COUNT'] < 100:
    names_to_replace.append(row['NAME'])

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# # Check to make sure binning was successful
application_df['NAME'].value_counts()

NAME
Other                                                                 25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CL

**'APPLICATION_TYPE' Binning**

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts_df = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts_df = app_type_counts_df.reset_index()
app_type_counts_df.columns = ['APPLICATION_TYPE', 'COUNT']
print(app_type_counts_df)

   APPLICATION_TYPE  COUNT
0                T3  27037
1                T4   1542
2                T6   1216
3                T5   1173
4               T19   1065
5                T8    737
6                T7    725
7               T10    528
8                T9    156
9               T13     66
10              T12     27
11               T2     16
12              T25      3
13              T14      3
14              T29      2
15              T15      2
16              T17      1


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

for index, row in app_type_counts_df.iterrows():
  if row['COUNT'] < 500:
    application_types_to_replace.append(row['APPLICATION_TYPE'])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

**'CLASSIFICATION' Cleaning**

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts_df = application_df['CLASSIFICATION'].value_counts()
class_counts_df = class_counts_df.reset_index()
class_counts_df.columns = ['CLASSIFICATION', 'COUNT']
print(class_counts_df)

   CLASSIFICATION  COUNT
0           C1000  17326
1           C2000   6074
2           C1200   4837
3           C3000   1918
4           C2100   1883
..            ...    ...
66          C4120      1
67          C8210      1
68          C2561      1
69          C4500      1
70          C2150      1

[71 rows x 2 columns]


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_to_keep = []

for index, row in class_counts_df.iterrows():
  if row['COUNT'] > 1:
    class_to_keep.append(row['CLASSIFICATION'])

application_df = application_df[application_df['CLASSIFICATION'].isin(class_to_keep)]

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for index, row in class_counts_df.iterrows():
  if row['COUNT'] < 1000:
    classifications_to_replace.append(row['CLASSIFICATION'])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

<ipython-input-14-ad8e99de34c5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
Name: count, dtype: int64

**Standardizing Data**

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_std_df = pd.get_dummies(application_df, columns=['INCOME_AMT','APPLICATION_TYPE',
                                                             'CLASSIFICATION',
                                                             'SPECIAL_CONSIDERATIONS', 'AFFILIATION', 'STATUS', 'USE_CASE', 'ORGANIZATION', 'NAME'])
application_std_df.head()

,ASK_AMT,IS_SUCCESSFUL,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,...,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TENNESSEE ORDER OF THE EASTERN STAR,NAME_THE UNITED STATES PONY CLUBS INC,NAME_TOASTMASTERS INTERNATIONAL,NAME_TOPS CLUB INC,NAME_UNITED STATES BOWLING CONGRESS INC,NAME_UNIVERSITY OF WYOMING,NAME_VETERANS OF FOREIGN WARS OF THE UNITED STATES AUXILIARY,NAME_WASHINGTON STATE GRANGE,NAME_WASHINGTON STATE UNIVERSITY
0,5000,1,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,108590,1,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,5000,0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6692,1,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# **Model Creation**

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_std_df.IS_SUCCESSFUL.values
X = application_std_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation="elu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation="elu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation="elu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                5600      
                                                                 
 dense_1 (Dense)             (None, 120)               9720      
                                                                 
 dense_2 (Dense)             (None, 120)               14520     
                                                                 
 dense_3 (Dense)             (None, 120)               14520     
                                                                 
 dense_4 (Dense)             (None, 1)                 121       
                                                                 
Total params: 44481 (173.75 KB)
Trainable params: 44481 (173.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.5154 - accuracy: 0.7434
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4978 - accuracy: 0.7539
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4926 - accuracy: 0.7568
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4910 - accuracy: 0.7592
Epoch 5/100
804/804 [==============================] - 5s 6ms/step - loss: 0.4899 - accuracy: 0.7597
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4882 - accuracy: 0.7588
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4869 - accuracy: 0.7608
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4870 - accuracy: 0.7604
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4858 - accuracy: 0.7609
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4863 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5146 - accuracy: 0.7512 - 576ms/epoch - 2ms/step
Loss: 0.514616847038269, Accuracy: 0.7511961460113525


In [ ]:
# Export our model to HDF5 file
nn.save("./AlphabetSoupCharityOptimized.h5")

from google.colab import files
files.download('./AlphabetSoupCharityOptimized.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>